# Remote Query Examples

This notebook demonstrates remote querying capabilities using `SPARQLWrapper`

SPARQLWrapper is a Python wrapper around a SPARQL service that gives the capability to
remotely execute queries on SPARQL endpoints. It simply invokes a SPARQLWrapper class
that requires an endpoint which you can pass a query to. The following are a few
examples that demonstate the capabilities:

### Query a single remote endpoint using `SPARQLWrapper`

In [1]:
import pandas as pd
from SPARQLWrapper import JSON, XML, SPARQLWrapper

This example creates a sparql class from the graph location: http://dbpedia.org/sparql.
In the setQuery method, you can pass a string that describes the query you want. Here,
we perform a SELECT query that returns the labels for the resource Asturias from
dbpedia:

In [3]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery(
    """
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?label
    WHERE { <http://dbpedia.org/resource/Asturias> rdfs:label ?label }
"""
)

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["label"]["value"])

Asturias
منطقة أستورياس
Asturien
Asturias
Asturies
Asturie
アストゥリアス州
Asturië (regio)
Asturia
Astúrias
Астурия
阿斯图里亚斯


The following example demonstates an ASK query:

In [4]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery(
    """
    ASK WHERE {
        <http://dbpedia.org/resource/Asturias> rdfs:label "Asturias"@es
    }
"""
)
sparql.setReturnFormat(XML)
results = sparql.query().convert()
print(results.toxml())

<?xml version="1.0" ?><sparql xmlns="http://www.w3.org/2005/sparql-results#" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.w3.org/2001/sw/DataAccess/rf1/result2.xsd">
 <head/>
 <boolean>true</boolean>
</sparql>


### Nested Query Example

A known issue with rdflib is that it does not support nested service calls in its query
body. An example of this can be seen in the
<a href="examples/FederatedQuery_Example.ipynb">Federated Query Examples</a> notebook.
We can utilize SPARQLWrapper to perform the same task. The following is a working
example of combining an in-memory graph with a remote graph that is called using nested
SERVICE clauses:

In [5]:
from rdflib import Graph, Literal, URIRef, namespace

WD = namespace.Namespace("https://www.wikidata.org/wiki/")
graph = Graph()
graph.add((WD.Q28792126, WD.example, Literal("Example")))
list(graph)

[(rdflib.term.URIRef('https://www.wikidata.org/wiki/Q28792126'),
  rdflib.term.URIRef('https://www.wikidata.org/wiki/example'),
  rdflib.term.Literal('Example'))]

In [6]:
sparql = SPARQLWrapper("https://query.wikidata.org/")
sparql.setQuery(
    """
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX bd: <http://www.bigdata.com/rdf#>

SELECT ?p ?o
WHERE {
    wd:Q28792126 ?p ?o .

    SELECT ?p ?o 
    WHERE 
    {
      BIND(wikibase:label AS ?p)
      wd:Q28792126 wdt:P31 wd:Q146
      service wikibase:label { 
          bd:serviceParam wikibase:language "en" . 
      }
    } 
    LIMIT 10
    
}
"""
)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

/Users/rhythmsyed/Desktop/GTRI/ipyradiant/envs/dev/lib/python3.7/site-packages/SPARQLWrapper/Wrapper.py:1346: RuntimeWarning: unknown response content type 'text/html' returning raw response...
  warnings.warn("unknown response content type '%s' returning raw response..." %(ct), RuntimeWarning)


In [7]:
len(results)

16754

In this example, we use SPARQLWrapper to query Wikidata via the Wikidata Query Service
and organize the output response using pandas:

In [8]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery(
    """
SELECT ?item ?itemLabel
WHERE
{
    ?item wdt:P31 wd:Q146 .
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
"""
)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [9]:
results_df = pd.io.json.json_normalize(results["results"]["bindings"])
results_df[["item.value", "itemLabel.value"]].head()

/Users/rhythmsyed/Desktop/GTRI/ipyradiant/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,item.value,itemLabel.value
0,http://www.wikidata.org/entity/Q28792126,Gli
1,http://www.wikidata.org/entity/Q30600575,Orlando
2,http://www.wikidata.org/entity/Q42442324,Kiisu Miisu
3,http://www.wikidata.org/entity/Q43260736,Paddles
4,http://www.wikidata.org/entity/Q48895080,Hamilton


For more examples and usage of SPARQLWrapper, please visit the Github repo found here:
https://github.com/RDFLib/sparqlwrapper